## Load Data

In [26]:
# Packages
import pandas as pd
import numpy as np

In [28]:
# path to data file
df_1 = pd.read_csv("/Users/murphy.chen/Desktop/data1030final_project/data/raw/Ad_table.csv", low_memory=False)
df_2 = pd.read_csv("/Users/murphy.chen/Desktop/data1030final_project/data/raw/Ad_table (extra).csv", low_memory=False)

df = pd.concat([df_1, df_2], ignore_index=True)

# check the combined data
print(df.shape)
print(list(df.columns))

(536510, 25)
['Maker', 'Genmodel', 'Genmodel_ID', 'Adv_ID', 'Adv_year', 'Adv_month', 'Color', 'Reg_year', 'Bodytype', 'Runned_Miles', 'Engin_size', 'Gearbox', 'Fuel_type', 'Price', 'Seat_num', 'Door_num', ' Genmodel_ID', 'Engine_power', 'Annual_Tax', 'Wheelbase', 'Height', 'Width', 'Length', 'Average_mpg', 'Top_speed']


## Data Cleaning

In [29]:
# start with keeping the necessary columns
columns_to_keep = ["Maker", 
                   "Genmodel", 
                   "Color", 
                   "Reg_year", 
                   "Bodytype", 
                   "Runned_Miles", 
                   "Engin_size", 
                   "Gearbox", 
                   "Fuel_type", 
                   "Price", 
                   "Seat_num",  
                   "Door_num"]

df = df[columns_to_keep].copy()

# check the new data columns
print(df.shape)
print(list(df.columns))

(536510, 12)
['Maker', 'Genmodel', 'Color', 'Reg_year', 'Bodytype', 'Runned_Miles', 'Engin_size', 'Gearbox', 'Fuel_type', 'Price', 'Seat_num', 'Door_num']


In [30]:
# clean and standardize the names
df.columns = (
    df.columns
    .str.strip()            
    .str.lower()             
    .str.replace(" ", "_")     
    .str.replace("-", "_") 
)

# check the cleaned column names
print(df.columns.tolist())

# rename some columns for better clarity
df = df.rename(columns={
    "runned_miles": "mileage",
    "engin_size": "engine_size",
    "genmodel": "model",
    "maker": "brand"
})

# check the renamed column names
print(df.columns.tolist())

['maker', 'genmodel', 'color', 'reg_year', 'bodytype', 'runned_miles', 'engin_size', 'gearbox', 'fuel_type', 'price', 'seat_num', 'door_num']
['brand', 'model', 'color', 'reg_year', 'bodytype', 'mileage', 'engine_size', 'gearbox', 'fuel_type', 'price', 'seat_num', 'door_num']


In [31]:
# check data types
print(df.dtypes)

brand           object
model           object
color           object
reg_year       float64
bodytype        object
mileage         object
engine_size     object
gearbox         object
fuel_type       object
price           object
seat_num       float64
door_num       float64
dtype: object


In [32]:
# clean numeric columns
def clean_numeric(series):
    return (
        series.astype(str)
        .str.replace(",", "", regex=False)
        .str.replace("km", "", case=False, regex=False)
        .str.replace("l", "", case=False, regex=False)
         .str.extract(r"(-?\d+\.?\d*)")[0]
        .astype(float)
    )

# apply the cleaning function to numeric columns
df["price"] = clean_numeric(df["price"])
df["mileage"] = clean_numeric(df["mileage"])
df["engine_size"] = clean_numeric(df["engine_size"])

for col in ["reg_year", "seat_num", "door_num"]:
    df[col] = pd.to_numeric(df[col], errors="coerce")

print(df[["price", "mileage", "engine_size", "reg_year", "seat_num", "door_num"]].head())


     price  mileage  engine_size  reg_year  seat_num  door_num
0  21500.0  60000.0          6.8    2000.0       5.0       4.0
1  28750.0  44000.0          6.8    2002.0       5.0       4.0
2  29999.0  55000.0          6.8    2002.0       5.0       4.0
3  34948.0  14000.0          6.8    2003.0       5.0       4.0
4  26555.0  61652.0          6.8    2003.0       5.0       4.0


In [36]:
# Count missing values per column
missing_count = df.isna().sum()
missing_fraction = missing_count / len(df)

missing_table = (
    pd.DataFrame({
        "missing_count": missing_count,
        "missing_fraction":(missing_fraction*100).round(2)
    })
    .query("missing_count > 0")
    .sort_values(by="missing_fraction", ascending=False)
)

print(missing_table)

             missing_count  missing_fraction
color                43750              8.15
seat_num             12948              2.41
door_num              9106              1.70
engine_size           4128              0.77
price                 2290              0.43
mileage               2110              0.39
bodytype              1908              0.36
fuel_type              818              0.15
gearbox                334              0.06
reg_year                14              0.00
